In [ ]:
author="Jumabek Alikhanov"
date = 'Nov 17,2019'

This is Final Project material for the "Deep Learning" class I took  

## Download the dataset from https://www.unb.ca/cic/datasets/ids-2017.html
We only need CSV files that is preprocessed and labeled for ML 

In [ ]:
#imports
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
%load_ext autoreload

In [ ]:
# load data
dataroot = 'MachineLearningCVE/'

In [ ]:
%autoreload 2
from preprocessing import load_data
X,y = load_data(dataroot) # reads csv file and returns np array of X,y -> of shape (N,D) and (N,1)

MachineLearningCVE/*.csv
['MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv', 'MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', 'MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv', 'MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 'MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv', 'MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv']
There are 2830743 flow records with 79 feature dimension
Data loaded.
Data preprocessing started...
Stripped column names
Dropped bad columns
There are 0 nan entries
Converted to numeric


## Data Imbalance
1. It is curucial to adress this issue in order to get decent performance
2. It also affects evaluation, we should calculate  `balanced accuracy`

In [ ]:
from preprocessing import balance_data, normalize
X = normalize(X)

In [ ]:
SEED=42

In [ ]:

%autoreload 2
from models import Classifier

def ensure_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def getClassifier(args,runs_dir=None):
    
    (method,optim,lr,reg,batch_size,input_dim,num_class,num_iters) = args
    if runs_dir is not None:
        ensure_dir(runs_dir)
    
    clf = Classifier(method,input_dim,num_class,lr=lr,reg=reg,num_epochs=num_iters,
                        batch_size=batch_size,runs_dir=runs_dir)
    return clf


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
def Kfold_evaluation(X,y,method='nn3',K=5,lr=1e-3,reg=1e-5,num_epochs=20):
    results = {}

    #hyper-params
    batch_size = 5120 # increasing batch size with more gpu added
    optim = 'Adam'

    N,input_dim = X.shape
    num_class = len(np.unique(y))
 
    skf = StratifiedKFold(n_splits=K,)
    for fold_index, (train_index,test_index) in enumerate(skf.split(X,y)): 
        print('---------------------------------------------')
        print('Fold #{}'.format(fold_index))    
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        
        X_train,y_train = balance_data(X_train,y_train,seed = SEED)
        
        classifier_args = (method,optim,lr,reg,batch_size,input_dim,num_class,num_epochs)
        config =  '{}/Kfold-{}th_run/optim_{}_lr_{}_reg_{}_bs_{}'.format(method,fold_index,optim,lr,reg,batch_size)
        runs_dir = join(dataroot,'runs',config)
    
        clf = getClassifier(classifier_args,runs_dir)
        
        clf.fit(X_train,y_train)
    
        pred = clf.predict(X_test,eval_mode=True)
        test_acc = metrics.balanced_accuracy_score(y_test,pred)*100
        print('balanced test acc: ',test_acc)
        results[fold_index]= (test_acc)
    return results

In [ ]:
#neural network with 3 layer
method = 'nn5'
K=5
results = Kfold_evaluation(X,y,method=method,K=K,lr=1e-3,reg=1e-5,num_epochs=20)

sum_test_acc = 0
for fold_index,res in results.items():
    (test_acc) = res
    sum_test_acc+= test_acc
print('{0:.2f}'.format(sum_test_acc/K))

---------------------------------------------
Fold #0


INFO:models:Classifier initialized with method nn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/nn5/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.6280
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2898
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2421
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2240
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2037
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.1845
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2058
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2019
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1918
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1816
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1787
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1842
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1796
DEBUG

Loaded MachineLearningCVE/runs/nn5/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 3 epochs and 102 mini batches
balanced test acc:  84.86105583548303
---------------------------------------------
Fold #1


INFO:models:Classifier initialized with method nn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/nn5/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.6766
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.3034
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2639
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2409
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2205
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2228
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.1952
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2264
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1885
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.2125
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1800
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1883
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1884
DEBUG

Loaded MachineLearningCVE/runs/nn5/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 2 epochs and 151 mini batches
balanced test acc:  87.38730502709997
---------------------------------------------
Fold #2


...

Loaded MachineLearningCVE/runs/nn5/Kfold-2th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 4 epochs and 203 mini batches
balanced test acc:  90.31020740930572
---------------------------------------------
Fold #3


INFO:models:Classifier initialized with method nn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/nn5/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.6800
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2963
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2565
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2671
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2236
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2730
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2088
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2289
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.2048
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1802
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1936
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1825
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1817
DEBUG

Loaded MachineLearningCVE/runs/nn5/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 3 epochs and 102 mini batches
balanced test acc:  86.47327250186615
---------------------------------------------
Fold #4


INFO:models:Classifier initialized with method nn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/nn5/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.6346
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.3089
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2410
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2249
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2162
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2074
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.1922
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2424
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1923
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1846
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1954
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1946
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1746
DEBUG

Loaded MachineLearningCVE/runs/nn5/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 2 epochs and 351 mini batches
balanced test acc:  79.12800402504438
85.63


In [ ]:
#neural network with 5 layer
method = 'cnn5'
K=5
results = Kfold_evaluation(X,y,method=method,K=K,lr=1e-3,reg=1e-5,num_epochs=20)

sum_test_acc = 0
for fold_index,res in results.items():
    (test_acc) = res
    sum_test_acc+= test_acc
print('{0:.2f}'.format(sum_test_acc/K))

---------------------------------------------
Fold #0


INFO:models:Classifier initialized with method cnn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn5/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.2606
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2551
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2026
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.1866
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.1859
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.1645
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.1743
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2102
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1770
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1549
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1640
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1713
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1540
DEB

Loaded MachineLearningCVE/runs/cnn5/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 5 epochs and 104 mini batches
balanced test acc:  84.05497950765717
---------------------------------------------
Fold #1


INFO:models:Classifier initialized with method cnn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn5/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.3015
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2754
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2403
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2087
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.1940
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.1727
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.1875
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2108
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1735
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1768
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1709
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1816
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1755
DEB

Loaded MachineLearningCVE/runs/cnn5/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 3 epochs and 102 mini batches
balanced test acc:  91.47832792596795
---------------------------------------------
Fold #2


...

Loaded MachineLearningCVE/runs/cnn5/Kfold-2th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 6 epochs and 305 mini batches
balanced test acc:  87.75651662651437
---------------------------------------------
Fold #3


INFO:models:Classifier initialized with method cnn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn5/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.2792
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2677
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2161
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2213
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2039
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.1785
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2017
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2057
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1905
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1731
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1746
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1766
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1607
DEB

Loaded MachineLearningCVE/runs/cnn5/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 4 epochs and 303 mini batches
balanced test acc:  87.02341103281196
---------------------------------------------
Fold #4


INFO:models:Classifier initialized with method cnn5, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn5/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.3017
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2782
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2438
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2095
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2135
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2006
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.1734
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.1998
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1745
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1690
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1766
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1688
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1659
DEB

Loaded MachineLearningCVE/runs/cnn5/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 4 epochs and 253 mini batches
balanced test acc:  85.48097058525335
87.16


In [ ]:
#neural network with 2 layer
method = 'cnn2'
K=5
results = Kfold_evaluation(X,y,method=method,K=K,lr=1e-3,reg=1e-5,num_epochs=20)

sum_test_acc = 0
for fold_index,res in results.items():
    (test_acc) = res
    sum_test_acc+= test_acc
print('{0:.2f}'.format(sum_test_acc/K))

---------------------------------------------
Fold #0


INFO:models:Classifier initialized with method cnn2, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn2/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.3505
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.3091
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2496
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2314
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2318
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.1951
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2231
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2159
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.1996
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1800
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1763
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1813
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1798
DEB

Loaded MachineLearningCVE/runs/cnn2/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 1 epochs and 300 mini batches
balanced test acc:  79.85577892969334
---------------------------------------------
Fold #1


INFO:models:Classifier initialized with method cnn2, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn2/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.3738
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.3042
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2804
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2402
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2394
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2111
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2302
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2106
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.2020
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1987
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.2002
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.2051
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1931
DEB

Loaded MachineLearningCVE/runs/cnn2/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 5 epochs and 304 mini batches
balanced test acc:  91.56018107282118
---------------------------------------------
Fold #2


...

Loaded MachineLearningCVE/runs/cnn2/Kfold-2th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 8 epochs and 57 mini batches
balanced test acc:  91.76181225506338
---------------------------------------------
Fold #3


INFO:models:Classifier initialized with method cnn2, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn2/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.3592
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.3094
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2686
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2623
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2423
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2181
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2615
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2295
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.2125
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.2016
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.1984
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1958
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1867
DEB

Loaded MachineLearningCVE/runs/cnn2/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 2 epochs and 151 mini batches
balanced test acc:  88.30745729161471
---------------------------------------------
Fold #4


INFO:models:Classifier initialized with method cnn2, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/cnn2/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 0.3395
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.2912
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.2650
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.2390
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.2255
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.2343
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.2002
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.2201
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.2010
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.1826
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.2114
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.1882
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.1935
DEB

Loaded MachineLearningCVE/runs/cnn2/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 2 epochs and 301 mini batches
balanced test acc:  84.141589236707
87.13


In [ ]:
#neural network with 3 layer
method = 'nn3'
K=5
results = Kfold_evaluation(X,y,method=method,K=K,lr=1e-3,reg=1e-5,num_epochs=20)

sum_test_acc = 0
for fold_index,res in results.items():
    (test_acc) = res
    sum_test_acc+= test_acc
print('{0:.2f}'.format(sum_test_acc/K))

---------------------------------------------
Fold #0


INFO:models:Classifier initialized with method nn3, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/nn3/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...


building NN3


DEBUG:models:Epoch [1/20], Step [50/398], Loss: 1.0634
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.6652
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.5548
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.4744
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.4520
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.4140
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.4078
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.4346
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.3864
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.3257
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.3233
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.3477
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.3225
DEBUG:models:Epoch [2/20], Step [301/398], Loss: 0.3094
DEBUG:models:Epoch [2/20], Step [351/398], Loss: 0.3141
DEBUG:models:Epoch [3/20], Step [2/398], Loss: 0.3363
DEBUG:models:Epoch [3/20], Step [52/398], Loss: 0.3258
DEBUG:models:Epoch [3/20], Step [102/398], Loss: 0.2962

Loaded MachineLearningCVE/runs/nn3/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 5 epochs and 354 mini batches
balanced test acc:  81.98621281537109
---------------------------------------------
Fold #1


...

Loaded MachineLearningCVE/runs/nn3/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 4 epochs and 353 mini batches
balanced test acc:  86.550616784367
---------------------------------------------
Fold #4


INFO:models:Classifier initialized with method nn3, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/nn3/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...


building NN3


DEBUG:models:Epoch [1/20], Step [50/398], Loss: 1.0672
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 0.6369
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 0.5170
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 0.4320
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 0.4075
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 0.3748
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 0.3616
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 0.3724
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 0.3449
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 0.3204
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 0.3445
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 0.3468
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 0.3178
DEBUG:models:Epoch [2/20], Step [301/398], Loss: 0.3099
DEBUG:models:Epoch [2/20], Step [351/398], Loss: 0.2928
DEBUG:models:Epoch [3/20], Step [2/398], Loss: 0.3019
DEBUG:models:Epoch [3/20], Step [52/398], Loss: 0.2696
DEBUG:models:Epoch [3/20], Step [102/398], Loss: 0.2973

Loaded MachineLearningCVE/runs/nn3/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 5 epochs and 354 mini batches
balanced test acc:  86.24194198056313
85.73


In [ ]:
#neural network with  softmax
method = 'softmax'
K=5
results = Kfold_evaluation(X,y,method=method,K=K,lr=1e-3,reg=1e-5,num_epochs=20)

sum_test_acc = 0
for fold_index,res in results.items():
    (test_acc) = res
    sum_test_acc+= test_acc
print('{0:.2f}'.format(sum_test_acc/K))

---------------------------------------------
Fold #0


INFO:models:Classifier initialized with method softmax, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/softmax/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 2.5434
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 2.3775
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 2.2526
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 2.1285
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 2.0302
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 1.9494
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 1.8808
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 1.7958
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 1.7579
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 1.6782
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 1.6250
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 1.5937
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 1.55

Loaded MachineLearningCVE/runs/softmax/Kfold-0th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 7 epochs and 356 mini batches
balanced test acc:  73.82532672957981
---------------------------------------------
Fold #1


INFO:models:Classifier initialized with method softmax, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/softmax/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 2.5311
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 2.3780
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 2.2457
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 2.1268
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 2.0459
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 1.9668
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 1.9071
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 1.8421
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 1.7872
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 1.7107
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 1.6944
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 1.6542
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 1.58

Loaded MachineLearningCVE/runs/softmax/Kfold-1th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 9 epochs and 108 mini batches
balanced test acc:  79.07317125426042
---------------------------------------------
Fold #2


...

Loaded MachineLearningCVE/runs/softmax/Kfold-2th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 19 epochs and 368 mini batches
balanced test acc:  77.87223704729863
---------------------------------------------
Fold #3


INFO:models:Classifier initialized with method softmax, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/softmax/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 2.5126
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 2.3446
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 2.2151
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 2.1176
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 2.0320
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 1.9502
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 1.8772
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 1.8184
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 1.7692
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 1.6941
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 1.6637
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 1.6256
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 1.55

Loaded MachineLearningCVE/runs/softmax/Kfold-3th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 19 epochs and 368 mini batches
balanced test acc:  81.22136833071558
---------------------------------------------
Fold #4


INFO:models:Classifier initialized with method softmax, input_dim 76, num_classes 15, num_epochs 20, batch_size 5120, lr 0.001000, reg 0.000010, runs_dir MachineLearningCVE/runs/softmax/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120
INFO:models:Starting training process...
DEBUG:models:Epoch [1/20], Step [50/398], Loss: 2.5374
DEBUG:models:Epoch [1/20], Step [100/398], Loss: 2.3664
DEBUG:models:Epoch [1/20], Step [150/398], Loss: 2.2394
DEBUG:models:Epoch [1/20], Step [200/398], Loss: 2.1237
DEBUG:models:Epoch [1/20], Step [250/398], Loss: 2.0426
DEBUG:models:Epoch [1/20], Step [300/398], Loss: 1.9462
DEBUG:models:Epoch [1/20], Step [350/398], Loss: 1.8922
DEBUG:models:Epoch [2/20], Step [1/398], Loss: 1.8265
DEBUG:models:Epoch [2/20], Step [51/398], Loss: 1.7807
DEBUG:models:Epoch [2/20], Step [101/398], Loss: 1.6928
DEBUG:models:Epoch [2/20], Step [151/398], Loss: 1.6532
DEBUG:models:Epoch [2/20], Step [201/398], Loss: 1.6136
DEBUG:models:Epoch [2/20], Step [251/398], Loss: 1.54

Loaded MachineLearningCVE/runs/softmax/Kfold-4th_run/optim_Adam_lr_0.001_reg_1e-05_bs_5120 model trained with batch_size = 5120, seen 19 epochs and 218 mini batches
balanced test acc:  69.37087913280486
76.27
